In [1]:
import pandas as pd

In [2]:
df_poi = pd.read_csv('data/poi/poi.csv', header=None, encoding='utf_8_sig').drop(0, axis=1)
df_poi.head(2)

,1,2,3,4,5,6,7
0,阿彪美食馆,嘉禾路104号香江花园113号,24.486259,118.118932,厦门市;思明区,美食;小吃快餐,0592-5558259
1,SM城市广场,嘉禾路468号,24.503625,118.121906,厦门市;湖里区,购物;综合商场,0592-5517933


In [3]:
df_poi[['Type','Subtype']] = df_poi[6].str.split(';', expand=True)
df_poi[['City','District']] = df_poi[5].str.split(';', expand=True)
df_poi.drop([5,6, 'City'], axis=1, inplace=True)
df_poi.head(2)

,1,2,3,4,7,Type,Subtype,District
0,阿彪美食馆,嘉禾路104号香江花园113号,24.486259,118.118932,0592-5558259,美食,小吃快餐,思明区
1,SM城市广场,嘉禾路468号,24.503625,118.121906,0592-5517933,购物,综合商场,湖里区


In [4]:
df_poi.columns = ['Name', 'Address', 'Latitude', 'Longitude', 'Phone', 'Type', 'Subtype', 'District']
df_poi.head()

,Name,Address,Latitude,Longitude,Phone,Type,Subtype,District
0,阿彪美食馆,嘉禾路104号香江花园113号,24.486259,118.118932,0592-5558259,美食,小吃快餐,思明区
1,SM城市广场,嘉禾路468号,24.503625,118.121906,0592-5517933,购物,综合商场,湖里区
2,SM新生活广场,嘉禾路399,24.502303,118.120023,0592-5589720,购物,综合商场,思明区
3,沃尔玛购物广场(SM城市广场店),嘉禾路468号SM城市广场B1层,24.503655,118.122076,0592-5511682; 0592-5569503,购物,超市,湖里区
4,麦当劳(SM城市广场店),嘉禾路468号SM城市广场F1,24.502856,118.122437,0592-5553189,美食,小吃快餐,湖里区


In [8]:
import geopandas as gpd

gdf = gpd.GeoDataFrame(
    df_poi, geometry=gpd.points_from_xy(df_poi['Latitude'],df_poi['Longitude'])
)

import geopandas as gpd
from shapely.geometry import Point

df_geometry = [Point(xy) for xy in zip(df_poi.Longitude, df_poi.Latitude)]
gdf = gpd.GeoDataFrame(df_poi, geometry=df_geometry)


In [9]:
import pandas as pd
import datetime
import numpy as np
import pytz   
import math
import geopandas as gpd
from shapely.geometry import Point,Polygon
from fiona.crs import from_epsg

In [10]:
lat_range = [24.2106, 25.0881]
long_range = [117.3944, 118.8913]

y = np.linspace(lat_range[0],lat_range[1],18)
x = np.linspace(long_range[0],long_range[1],18)

grid = []
for m,i in enumerate(x[:17]):
    for k,j in enumerate(y[:17]):
        cord = np.array([i,j,x[m+1],j,x[m+1],y[k+1],i,y[k+1]]).reshape(4,2)
        grid.append(Polygon(cord))


area = gpd.GeoDataFrame(grid)
area = area.reset_index()
area.columns=['index','geometry']
area.crs={'init':'epsg:4326'}


t_gdf = gpd.sjoin(gdf, area, op='within', how='left')
t_gdf.head()


/Users/arthur/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/Users/arthur/opt/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != +init=epsg:4326 +type=crs)
  "(%s != %s)" % (left_df.crs, right_df.crs)


,Name,Address,Latitude,Longitude,Phone,Type,Subtype,District,geometry,index_right,index
0,阿彪美食馆,嘉禾路104号香江花园113号,24.486259,118.118932,0592-5558259,美食,小吃快餐,思明区,POINT (118.11893 24.48626),141.0,141.0
1,SM城市广场,嘉禾路468号,24.503625,118.121906,0592-5517933,购物,综合商场,湖里区,POINT (118.12191 24.50363),141.0,141.0
2,SM新生活广场,嘉禾路399,24.502303,118.120023,0592-5589720,购物,综合商场,思明区,POINT (118.12002 24.50230),141.0,141.0
3,沃尔玛购物广场(SM城市广场店),嘉禾路468号SM城市广场B1层,24.503655,118.122076,0592-5511682; 0592-5569503,购物,超市,湖里区,POINT (118.12208 24.50366),141.0,141.0
4,麦当劳(SM城市广场店),嘉禾路468号SM城市广场F1,24.502856,118.122437,0592-5553189,美食,小吃快餐,湖里区,POINT (118.12244 24.50286),141.0,141.0


In [28]:
t_gdf.groupby(['Type'])['index'].count()

Type
交通运输       79
休闲娱乐    10524
公司企业    59006
其它         19
医疗保健     6489
地名地址     2156
基础设施    25408
建筑房产    48733
教育学校     9478
文化场馆     2550
旅游服务     1059
机构团体     6478
汽车      14652
生活服务    32422
美食      54475
购物      86767
运动健身     2788
酒店宾馆     9882
银行金融     7671
Name: index, dtype: int64

In [49]:
poi_types = ['休闲娱乐', '基础设施', '文化场馆', '美食', '购物', '酒店宾馆']
t_df = t_gdf.loc[t_gdf['Type'].isin(poi_types)]

In [62]:
stat = area.loc[:,['index']]
stat = pd.merge(stat, t_df.groupby('index')['Type'].count().to_frame().reset_index(),
               on='index', how='outer')
stat.columns = ['index', 'count']
stat.fillna(0,inplace=True) 

In [63]:
stat.head()

,index,count
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [64]:
for condition in poi_types:
    stat = pd.merge(stat, t_df[t_df['Type']==condition].groupby('index')['Type'].count().to_frame().reset_index(),
               on='index', how='outer')
stat.columns = ['index', 'count'] + poi_types
stat.fillna(0,inplace=True) 
stat.head()

,index,count,休闲娱乐,基础设施,文化场馆,美食,购物,酒店宾馆
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
sorted(stat['count'].unique())[-10:]

[5953.0,
 6369.0,
 8709.0,
 8916.0,
 10141.0,
 11653.0,
 12278.0,
 17280.0,
 20745.0,
 57908.0]

In [67]:
stat.to_csv('data/poi/requ.csv',index=False)

In [47]:
area.columns

Index(['index', 'geometry'], dtype='object')

In [48]:
grid_id = stat[stat['count'] == 116864.0]['index'].values[0]
area[area['index'] == grid_id]

,index,geometry
141,141,"POLYGON ((118.09882 24.46869, 118.18688 24.468..."


In [108]:
l_l_long_lat = df_poi[['Longitude', 'Latitude']].to_numpy().tolist()

In [ ]:
import numpy as np
from sklearn.cluster import Birch


brc = Birch(branching_factor=50, n_clusters=None, threshold=0.0001)
brc.fit(l_l_long_lat)
labels = brc.predict(l_l_long_lat)
df_poi['cluster'] = labels
len(df_poi['cluster'].unique())

In [72]:
# pip install weibo
from weibo import Client
from weibopy import WeiboClient

In [83]:
API_KEY=494609143
API_SECRET="a2c5a854908953a9af53c4ef95c2829a"
REDIRECT_URI="http://127.0.0.1/callback"
TOKEN = {'access_token': '2.00QSUfKI0p71TX112616462e0RY8Hd',
            'remind_in': '157679999',
            'uid': '7486713916',
            'isRealName': 'true',
            'expires_at': 1753993559}



In [80]:
c = Client(API_KEY, API_SECRET, REDIRECT_URI)
c.authorize_url

'https://api.weibo.com/oauth2/authorize?client_id=494609143&redirect_uri=http%3A%2F%2F127.0.0.1%2Fcallback'

In [81]:
c.set_code('f06f0546307f3fadf1c4f88ea08bbac5')

In [82]:
t = c.token
t

{'access_token': '2.00QSUfKI0p71TX112616462e0RY8Hd',
 'remind_in': '157679999',
 'uid': '7486713916',
 'isRealName': 'true',
 'expires_at': 1753993559}

In [100]:
c = Client(API_KEY, API_SECRET, REDIRECT_URI, TOKEN)

In [101]:
API_POI_TIMELINE = 'place/poi_timeline'
API_POI_ID = 'place/pois/show'
API_POI_NEARBY = 'place/nearby/pois/biz'

In [102]:
lat_long = [24.503625, 118.121906]

In [103]:
poi_nb = c.get(API_POI_NEARBY, lat=24.503625, long=118.121906)
poi_nb

https://api.weibo.com/2/place/nearby/pois/biz.json
<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html>
<head><title>404 Not Found</title></head>
<body bgcolor="white">
<h1>404 Not Found</h1>
<p>The requested URL was not found on this server. Sorry for the inconvenience.<br/>
Please report this message and include the following information to us.<br/>
Thank you very much!</p>
<table>
<tr>
<td>URL:</td>
<td>http://i.api.place.weibo.cn/place/nearby/pois/biz.json?access_token=2.00QSUfKI0p71TX112616462e0RY8Hd&amp;lat=24.503625&amp;long=118.121906</td>
</tr>
<tr>
<td>Server:</td>
<td>lbs-172-16-190-55.dbl.intra.weibo.cn</td>
</tr>
<tr>
<td>Date:</td>
<td>2020/08/02 04:32:52</td>
</tr>
</table>
<hr/>Powered by Tengine</body>
</html>



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
poi_result = c.get(API_POI_TIMELINE, poiid=, since_id=0, max_id=0, count=20, page=1, base_app=0 )

In [99]:
try:
    from urllib.parse import urlencode
except ImportError:
    from urllib import urlencode

import json
import time

import requests


class Client(object):
    def __init__(self, api_key, api_secret, redirect_uri, token=None,
                 username=None, password=None):
        # const define
        self.site = 'https://api.weibo.com/'
        self.authorization_url = self.site + 'oauth2/authorize'
        self.token_url = self.site + 'oauth2/access_token'
        self.api_url = self.site + '2/'

        # init basic info
        self.client_id = api_key
        self.client_secret = api_secret
        self.redirect_uri = redirect_uri

        self.session = requests.session()
        if username and password:
            self.session.auth = username, password

        # activate client directly if given token
        if token:
            self.set_token(token)

    @property
    def authorize_url(self):
        params = {
            'client_id': self.client_id,
            'redirect_uri': self.redirect_uri
        }
        return "{0}?{1}".format(self.authorization_url, urlencode(params))

    @property
    def alive(self):
        if self.expires_at:
            return self.expires_at > time.time()
        else:
            return False

    def set_code(self, authorization_code):
        """Activate client by authorization_code.
        """
        params = {
            'client_id': self.client_id,
            'client_secret': self.client_secret,
            'grant_type': 'authorization_code',
            'code': authorization_code,
            'redirect_uri': self.redirect_uri
        }
        res = requests.post(self.token_url, data=params)
        token = json.loads(res.text)
        self._assert_error(token)

        token[u'expires_at'] = int(time.time()) + int(token.pop(u'expires_in'))
        self.set_token(token)

    def set_token(self, token):
        """Directly activate client by access_token.
        """
        self.token = token

        self.uid = token['uid']
        self.access_token = token['access_token']
        self.expires_at = token['expires_at']

        self.session.params = {'access_token': self.access_token}

    def _assert_error(self, d):
        """Assert if json response is error.
        """
        if 'error_code' in d and 'error' in d:
            raise RuntimeError("{0} {1}".format(
                d.get("error_code", ""), d.get("error", "")))

    def get(self, uri, **kwargs):
        """Request resource by get method.
        """
        url = "{0}{1}.json".format(self.api_url, uri)

        # for username/password client auth
        if self.session.auth:
            kwargs['source'] = self.client_id

        
        print(url)
        t_str = self.session.get(url, params=kwargs).text
        print(t_str)
        res = json.loads(t_str)
        self._assert_error(res)
        return res

    def post(self, uri, **kwargs):
        """Request resource by post method.
        """
        url = "{0}{1}.json".format(self.api_url, uri)

        # for username/password client auth
        if self.session.auth:
            kwargs['source'] = self.client_id

        if "pic" not in kwargs:
            res = json.loads(self.session.post(url, data=kwargs).text)
        else:
            files = {"pic": kwargs.pop("pic")}
            res = json.loads(self.session.post(url,
                                               data=kwargs,
                                               files=files).text)
        self._assert_error(res)
        return res